In [ ]:
pip install -U langchain langchain-community langchain-core langchain-classic langchain_chroma langchain-huggingface langchain-google-genai

In [ ]:
import os
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

creates the mathematical vectors

In [51]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Point to your Vector DB

In [52]:
vector_db = Chroma(
    persist_directory="./my_vectordb", 
    embedding_function=embeddings,
    collection_name="office_docs"
)

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-flash-latest", google_api_key="Key")

Prompt using the {context} variable

In [86]:
prompt = ChatPromptTemplate.from_template("""
Answer the question using the provided context. If the information is not in the context, use your general knowledge to provide a helpful answer, but clearly state if you are using outside information.:

<context>
{context}
</context>

Question: {input}
""")

Creates the chains

In [87]:
combine_docs_chain = create_stuff_documents_chain(llm,prompt)

Creates the retriever

In [88]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [89]:
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [90]:
response = rag_chain.invoke({"input": "What is the office location? and in which country the city is located?"})
print(response["answer"])

The office location is **Bangalore**.

Bangalore is located in **India**. (This information regarding the country is based on general knowledge, as it was not explicitly mentioned in the provided context.)
